In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-06-04 23:17:31--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   179MB/s    in 0.5s    

2021-06-04 23:17:32 (179 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem


import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import neighbors, svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC, SVR
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


import matplotlib
#matplotlib.use('GTKAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
%matplotlib inline
matplotlib.rcParams.update({'font.size': 18})

In [ ]:
data_AUC = pd.read_csv('AUC_final.csv')
data_CL_max = pd.read_csv('CL_max_final.csv')
data_Cmax = pd.read_csv('Cmax_final.csv')
data_F = pd.read_csv('F_final.csv')
data_IC_50 = pd.read_csv('IC_50_final.csv')
data_LD_50 = pd.read_csv('LD_50_final.csv')
data_PPB = pd.read_csv('PPB_final.csv')
data_T_half = pd.read_csv('T_1:2_final.csv')
data_Vdss = pd.read_csv('Vdss_final.csv')


In [ ]:
df = data_T_half

In [ ]:
df

,chembl_id,organism,canonical_smiles,T1/2 /hr
0,CHEMBL99828,Rattus norvegicus,Cc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(F)cc2)s1,19.40000
1,CHEMBL9946,Rattus norvegicus,CCCCN(CC)c1nc(C)nc2c1c(C)cn2-c1c(C)cc(C)cc1C,1.50000
2,CHEMBL99214,Rattus norvegicus,C[C@@H]1[C@H](OC(=O)N[C@@H](CSc2ccccc2)[C@H](O...,2.53300
3,CHEMBL99066,Rattus norvegicus,CC(C)c1ccccc1Sc1ccc(/C=C/C(=O)N2CCCC(C(=O)O)C2...,0.40000
4,CHEMBL98936,Rattus norvegicus,O=C(N[C@@H](Cc1ccccc1)C[C@H](O)[C@H](Cc1ccccc1...,0.69000
...,...,...,...,...
17232,CHEMBL101083,Rattus norvegicus,CC1(c2ccc(OCCN3CCCCC3)cc2)c2ccc(O)cc2CCN1c1ccc...,7.10000
17233,CHEMBL1007,Rattus norvegicus,CC(C)C[C@H](NC(=O)CNC(=O)[C@H](Cc1ccc(O)cc1)NC...,2.63000
17234,CHEMBL100379,Rattus norvegicus,C[C@H](NC(=O)/C=C/c1ccc(F)cc1)c1ccc(F)c(N2CCOC...,3.50000
17235,CHEMBL100367,Rattus norvegicus,O=C(O)C1CCCN(C(=O)/C=C/c2ccc(Sc3ccc4c(c3)OCCO4...,2.30000


In [ ]:
df = df[df['T1/2 /hr'] < 1]

In [ ]:
df

,chembl_id,organism,canonical_smiles,T1/2 /hr
3,CHEMBL99066,Rattus norvegicus,CC(C)c1ccccc1Sc1ccc(/C=C/C(=O)N2CCCC(C(=O)O)C2...,0.40000
4,CHEMBL98936,Rattus norvegicus,O=C(N[C@@H](Cc1ccccc1)C[C@H](O)[C@H](Cc1ccccc1...,0.69000
7,CHEMBL982,Mus musculus,C=C1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3C...,0.33330
8,CHEMBL9806,Mus musculus,CS(=O)(=O)Nc1ccc(Nc2c3ccccc3nc3ccccc23)cc1,0.91670
9,CHEMBL98,Mus musculus,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0.38000
...,...,...,...,...
17219,CHEMBL10257,Rattus norvegicus,COc1ccc(C#Cc2ccc(S(=O)(=O)NC(CC#Cc3ccccc3)C(=O...,0.70000
17222,CHEMBL102299,Mus musculus,Cc1ccccc1NC(=O)Nc1ccc(CC(=O)N[C@@H](CC(C)C)C(=...,0.80000
17226,CHEMBL101874,Bos taurus,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@H](CC(C)...,0.16670
17230,CHEMBL1014,Rattus norvegicus,CCOc1nc2cccc(C(=O)OC(C)OC(=O)OC3CCCCC3)c2n1Cc1...,0.08200


In [ ]:
from rdkit.Chem import Descriptors

def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        #desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([#desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_1(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_NumHAcceptors = Descriptors.NumHAcceptors(mol)
        desc_NumHDonors = Descriptors.NumHDonors(mol)
        desc_NumHeteroatoms = Descriptors.NumHeteroatoms(mol)
           
        row = np.array([desc_NumHAcceptors,
                        desc_NumHDonors,
                        desc_NumHeteroatoms])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["NumHAcceptors","NumHDonors","NumHeteroatoms"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_2(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolMR = Descriptors.MolMR(mol)
        desc_HeavyAtomCount = Descriptors.HeavyAtomCount(mol)
        desc_HeavyAtomMolwt = Descriptors.HeavyAtomMolWt(mol)
           
        row = np.array([desc_MolMR,
                        desc_HeavyAtomCount,
                        desc_HeavyAtomMolwt])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolMR","HeavyAtomCount","HeavyAtomMolwt"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_3(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_NOCount = Descriptors.NOCount(mol)
        #desc_NHOHCount = Descriptors.NHOHCount(mol)
        desc_NumValenceElectrons = Descriptors.NumValenceElectrons(mol)
           
        row = np.array([desc_NOCount,
                        #desc_NHOHCount,
                        desc_NumValenceElectrons])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["NOCount","NumValenceElectrons"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_4(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_RingCount = Descriptors.RingCount(mol)
        desc_qed = Descriptors.qed(mol)
        desc_MaxAbsEStateIndex = Descriptors.MaxAbsEStateIndex(mol)
           
        row = np.array([desc_RingCount,
                        desc_qed,
                        desc_MaxAbsEStateIndex])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["RingCount","qed","MaxAbsEStateIndex"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_5(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_TPSA = Descriptors.TPSA(mol)
        desc_LabuteASA = Descriptors.LabuteASA(mol)
        desc_PEOE_VSA1 = Descriptors.PEOE_VSA1(mol)
           
        row = np.array([desc_TPSA,
                        desc_LabuteASA,
                        desc_PEOE_VSA1])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["TPSA","LabuteASA","PEOE_VSA1"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors



def generate_7(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_FractionCSP3 = Descriptors.FractionCSP3(mol)
        desc_MaxPartialCharge = Descriptors.MaxPartialCharge(mol)
        desc_MinPartialCharge = Descriptors.MinPartialCharge(mol)
           
        row = np.array([desc_FractionCSP3,
                        desc_MaxPartialCharge,
                        desc_MinPartialCharge])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["FractionCSP3","MaxPartialCharge","MinPartialCharge"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_8(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_FpDensityMorgan1 = Descriptors.FpDensityMorgan1(mol)
        desc_FpDensityMorgan2 = Descriptors.FpDensityMorgan2(mol)
        desc_FpDensityMorgan3 = Descriptors.FpDensityMorgan3(mol)
           
        row = np.array([desc_FpDensityMorgan1,
                        desc_FpDensityMorgan2,
                        desc_FpDensityMorgan3])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["FpDensityMorgan1","FpDensityMorgan2","FpDensityMorgan3"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_9(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_Chi3n = Descriptors.Chi3n(mol)
        desc_Chi3v = Descriptors.Chi3v(mol)
        #desc_Chi4v = Descriptors.Chi4v(mol)
           
        row = np.array([desc_Chi3n,
                        desc_Chi3v])
                        #desc_Chi4v])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["Chi3n","Chi3v"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_10(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_Kappa3 = Descriptors.Kappa3(mol)
        desc_PEOE_VSA10 = Descriptors.PEOE_VSA10(mol)
        desc_SMR_VSA4 = Descriptors.SMR_VSA4(mol)
           
        row = np.array([desc_Kappa3,
                        desc_PEOE_VSA10,
                        desc_SMR_VSA4])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["Kappa3","PEOE_VSA10","SMR_VSA4"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_11(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_PEOE_VSA12 = Descriptors.PEOE_VSA12(mol)
        desc_PEOE_VSA2 = Descriptors.PEOE_VSA2(mol)
        desc_PEOE_VSA5 = Descriptors.PEOE_VSA5(mol)
           
        row = np.array([desc_PEOE_VSA12,
                        desc_PEOE_VSA2,
                        desc_PEOE_VSA5])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["PEOE_VSA12","PEOE_VSA2","PEOE_VSA5"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_11(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_SMR_VSA10 = Descriptors.SMR_VSA10(mol)
        desc_PEOE_VSA9 = Descriptors.PEOE_VSA9(mol)
        desc_PEOE_VSA6 = Descriptors.PEOE_VSA6(mol)
           
        row = np.array([desc_SMR_VSA10,
                        desc_PEOE_VSA9,
                        desc_PEOE_VSA6])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["SMR_VSA10","PEOE_VSA9","PEOE_VSA6"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors




def generate_14(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_SlogP_VSA4 = Descriptors.SlogP_VSA4(mol)
        desc_SlogP_VSA5 = Descriptors.SlogP_VSA5(mol)
        desc_SlogP_VSA2 = Descriptors.SlogP_VSA2(mol)
           
        row = np.array([desc_SlogP_VSA4,
                        desc_SlogP_VSA5,
                        desc_SlogP_VSA2])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["SlogP_VSA4","SlogP_VSA5","SlogP_VSA2"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_15(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_SlogP_VSA7 = Descriptors.SlogP_VSA7(mol)
        
        desc_Estate_VSA1 = Descriptors.EState_VSA1(mol)
           
        row = np.array([desc_SlogP_VSA7,
                        
                        desc_Estate_VSA1])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["SlogP_VSA7","Estate_VSA1"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_16(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_Estate_VSA10 = Descriptors.EState_VSA10(mol)
        desc_Estate_VSA2 = Descriptors.EState_VSA2(mol)
        desc_Estate_VSA3 = Descriptors.EState_VSA3(mol)
           
        row = np.array([desc_Estate_VSA10,
                        desc_Estate_VSA2,
                        desc_Estate_VSA3])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["Estate_VSA10","Estate_VSA2","Estate_VSA3"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_17(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_Estate_VSA4 = Descriptors.EState_VSA4(mol)
        desc_Estate_VSA7 = Descriptors.EState_VSA7(mol)
        desc_Estate_VSA8 = Descriptors.EState_VSA8(mol)
           
        row = np.array([desc_Estate_VSA4,
                        desc_Estate_VSA7,
                        desc_Estate_VSA8])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["Estate_VSA4","Estate_VSA7","Estate_VSA8"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

def generate_18(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_EState_VSA9 = Descriptors.EState_VSA9(mol)
        desc_VSA_Estate1 = Descriptors.VSA_EState1(mol)
        
        row = np.array([desc_EState_VSA9,
                        desc_VSA_Estate1
                        ])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["EState_VSA9","VSA_Estate1"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors



def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [ ]:
df1 = generate(df.canonical_smiles)

In [ ]:
df2 = generate_1(df.canonical_smiles)

In [ ]:
df3 = generate_2(df.canonical_smiles)

In [ ]:
df4 = generate_3(df.canonical_smiles)

In [ ]:
df5 = generate_4(df.canonical_smiles)

In [ ]:
df6 = generate_5(df.canonical_smiles)

In [ ]:
df8 = generate_7(df.canonical_smiles)

In [ ]:
df9 = generate_8(df.canonical_smiles)

In [ ]:
df10 = generate_9(df.canonical_smiles)

In [ ]:
df11 = generate_10(df.canonical_smiles)

In [ ]:
df12 = generate_11(df.canonical_smiles)

In [ ]:
# df13 = generate_12(df.canonical_smiles)

In [ ]:
# df14 = generate_13(df.canonical_smiles)

In [ ]:
df15 = generate_14(df.canonical_smiles)

In [ ]:
df16 = generate_15(df.canonical_smiles)

In [ ]:
df17 = generate_16(df.canonical_smiles)

In [ ]:
df18 = generate_17(df.canonical_smiles)

In [ ]:
df19 = generate_18(df.canonical_smiles)

In [ ]:
mol_list= []
for element in df.canonical_smiles:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])


In [ ]:
X = pd.concat([df1,df2,df3,df4,df5,df6,df8,df9,df10,df11,df12,df15,df16,df17,df18,df19], axis=1)
X

,MolWt,NumRotatableBonds,NumHAcceptors,NumHDonors,NumHeteroatoms,MolMR,HeavyAtomCount,HeavyAtomMolwt,NOCount,NumValenceElectrons,RingCount,qed,MaxAbsEStateIndex,TPSA,LabuteASA,PEOE_VSA1,FractionCSP3,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,Chi3n,Chi3v,Kappa3,PEOE_VSA10,SMR_VSA4,SMR_VSA10,PEOE_VSA9,PEOE_VSA6,SlogP_VSA4,SlogP_VSA5,SlogP_VSA2,SlogP_VSA7,Estate_VSA1,Estate_VSA10,Estate_VSA2,Estate_VSA3,Estate_VSA4,Estate_VSA7,Estate_VSA8,EState_VSA9,VSA_Estate1
0,454.548,7.0,5,1,8,123.7242,32.0,428.340,7,168,3.0,0.350287,12.475971,100.75,190.825566,10.006437,0.333333,0.307797,-0.480977,1.218750,1.968750,2.656250,5.441291,6.598123,5.761070,0.000000,5.917906,35.401776,15.736701,49.874828,16.032224,43.733927,39.895746,0.0,16.810522,24.809920,18.139322,35.763241,10.458935,24.265468,13.847474,0.000000,0.000000
1,567.664,11.0,9,3,11,146.9692,40.0,534.400,10,212,5.0,0.320161,12.989617,128.24,236.174313,34.687556,0.413793,0.413188,-0.443275,1.000000,1.725000,2.425000,7.085612,7.733788,7.785177,6.103966,5.917906,23.523266,42.984860,72.000457,5.917906,23.968546,66.072391,0.0,36.478095,14.695602,25.235636,24.511733,17.547725,60.663671,15.617556,18.947452,22.165829
2,339.435,2.0,4,2,4,93.9306,25.0,314.235,4,132,6.0,0.813194,12.079797,52.93,147.648622,14.949918,0.619048,0.165687,-0.504246,1.280000,2.120000,2.920000,7.711452,7.711452,1.570641,6.103966,5.917906,0.000000,11.016041,12.645303,5.917906,43.231011,45.949335,0.0,11.016041,10.213055,17.895319,5.749512,55.827261,6.066367,11.478845,4.736863,6.287456
3,363.442,4.0,4,2,6,107.4402,26.0,346.306,5,130,4.0,0.523688,11.335803,71.09,151.296917,5.316789,0.050000,0.229319,-0.354417,0.730769,1.230769,1.769231,4.045163,4.651656,3.316732,0.000000,0.000000,48.891300,22.976557,36.398202,0.000000,0.000000,19.657545,0.0,10.023291,8.417797,0.000000,5.687386,39.436392,60.663671,10.038883,4.983979,25.137508
4,264.325,8.0,3,3,5,72.7029,19.0,244.165,5,104,1.0,0.383220,11.591415,78.43,112.521981,5.316789,0.428571,0.242829,-0.326098,0.894737,1.473684,2.052632,2.739924,2.739924,7.205209,0.000000,0.000000,17.501746,0.000000,31.040744,0.000000,38.524930,17.021612,0.0,0.000000,9.589074,11.814359,12.841643,31.370673,30.331835,5.316789,5.207253,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5791,459.523,6.0,4,2,7,125.0983,33.0,438.355,6,166,3.0,0.552246,12.660696,92.70,192.843518,9.843390,0.115385,0.322235,-0.496767,0.878788,1.424242,1.969697,4.514018,6.001605,6.979242,11.791353,0.000000,15.992596,12.005281,41.880838,23.681737,23.111176,32.645268,0.0,22.034437,18.318862,11.316305,5.563451,16.876415,30.331835,28.403832,4.736863,32.633011
5792,522.646,10.0,4,4,9,146.9569,38.0,484.342,9,204,3.0,0.363509,13.385094,127.84,223.536424,25.956803,0.448276,0.323188,-0.481174,1.000000,1.710526,2.368421,6.630634,6.630634,8.606306,6.041841,5.917906,35.189552,12.841643,44.179310,12.841643,57.078485,52.449654,0.0,12.011146,24.284676,42.646864,25.073786,29.655933,45.036680,15.950366,0.000000,0.000000
5793,813.010,21.0,10,7,17,216.6869,58.0,748.498,17,322,2.0,0.090946,13.846600,238.81,341.252372,46.358436,0.609756,0.407819,-0.467476,0.706897,1.293103,1.827586,9.192662,9.192662,16.549756,41.852096,17.753718,41.598444,19.131046,78.295080,17.753718,93.336395,100.528294,0.0,89.368446,33.561760,31.098277,12.114750,5.563451,26.837579,41.868689,9.473726,10.260239
5794,610.671,10.0,11,1,12,164.3702,45.0,576.399,12,232,6.0,0.140910,13.390098,143.34,259.212237,18.947452,0.333333,0.511146,-0.464936,0.955556,1.733333,2.466667,7.694214,7.694214,6.792704,6.103966,0.000000,23.158072,29.748491,61.018125,0.000000,61.873023,61.300758,0.0,18.414698,9.589074,11.667418,36.019909,54.357914,66.088140,25.608112,18.947452,23.894188


In [ ]:
y = df.iloc[:,3]

In [ ]:

X_train = StandardScaler().fit(X_train).transform(X_train)
X_test = StandardScaler().fit(X_test).transform(X_test)

regressor=SVR(kernel='rbf',verbose=False, C = 1, gamma = 0.2)
regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test))

Mean squared error (MSE): 0.06
Coefficient of determination (R^2): 0.20
